# Getting the Git

In [209]:
import os
%matplotlib inline
import pandas as pd
import numpy as np
import scipy as sp
import timeit
import seaborn as sns
from joblib import Parallel, delayed
import multiprocessing
import matplotlib as mpl
import time
import csv
import math
from time import sleep
import sys
#os.environ['PATH'] += ";C\Program Files\Git\\bin"

'''# ---------------- Old Toolbar Updated -------------------------#
toolbar_width = 40

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1)) 
'''

from time import sleep
import sys

def progbar(i,j):
    sys.stdout.write('\r')
    k = round((i / j) * 100,2)
    sys.stdout.write("%d%%" % k)
    sys.stdout.flush()
    sleep(0.01)

In [ ]:
#%%bash
#rm -rf fec-data
#git clone git@github.com:jbisbee1/fec-data.git

In [ ]:
#%%bash
#git log

# Opening the full data

In [2]:
recipients = pd.read_stata("../../rawrecipients.dta")
recipients.head()

,raw_recipient,n
0,! A+ TUTOR U LEARNIGN CENTERS OF CENTRAL FLORI...,1
1,!1T Family Dental,3
2,!2 Technologies,1
3,"""33"" Ranch",1
4,"""Concept"" Employees Of",1


# Fuzzy matching on firm names

I use the fuzzywuzzy package and run through each firm name, comparing it to the remaining 5.6 million in the dataset. 

In [ ]:
#%%bash
#pip install fuzzywuzzy

In [ ]:
#%%bash
#pip install "C:/Users/jhb362/Dropbox/Programs/python_Levenshtein-0.12.0-cp35-none-win_amd64.whl"

In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [11]:
recclean = recipients['raw_recipient'][:10000]
recclean.head()
str(sys.argv)
int(sys.argv[1])

0    ! A+ TUTOR U LEARNIGN CENTERS OF CENTRAL FLORI...
1                                    !1T Family Dental
2                                      !2 Technologies
3                                           "33" Ranch
4                               "Concept" Employees Of
Name: raw_recipient, dtype: object

Each of the firm names is compared against a master list which is stripped of any non-string types (although I'm not sure why any would be in there, there seem to be a few.) I also drop duplicates but, again, I don't think there should be too many in there, given all the work already conducted using Stata.

In [12]:
uniques = recclean.drop_duplicates()
select = [isinstance(e,str) for e in uniques]
uniques2 = uniques[select]

In [13]:
select = [isinstance(e,str) for e in recclean]
recclean2 = recclean[select]
print(len(recclean2), len(uniques2))

10000 10000


Confirmed that there are no duplicates in the list. 

Now we're off to the races. This is going to take a goddamned long time so smoke 'em if you got 'em. We go through every firm name in the full list of 5.6m and compare the string to all of the others using the fuzzywuzzy method "process". 

Whoa...too long. I'm going to try and run this on the cluster. No sense in gumming up my machine.

In [28]:
["test",[x[0] for x in process.extractBests("test",["testing","testers","test akk","abs test"],scorer=fuzz.token_set_ratio,score_cutoff=90)]]

['test', ['test akk', 'abs test']]

In [77]:
pairings=[]
counter = 0
argstest = 0

for rec in recclean2[argstest:argstest+1000]:
    pairings.append([rec,[x[0] for x in process.extractBests(rec,\
                                                             uniques2,\
                                                             scorer=fuzz.token_set_ratio,\
                                                             score_cutoff=90)]])
    progbar(counter,1000-1)
    counter += 1

100%

In [194]:
new = []
for j in range(len(pairings)):
    new.append([pairings[j][0],[i for i in pairings[j][1] if pairings[j][0] != i]])
mask = [new[i] for i in range(len(new)) if len(new[i][1]) > 0]

new2 = []
for j in range(len(mask)):
    new2.append([mask[j][0],[x[0] for x in process.extractBests(mask[j][0],\
                                                                mask[j][1],\
                                                                scorer = fuzz.token_sort_ratio,\
                                                                score_cutoff=85)]])
mask2 = [new2[i] for i in range(len(new2)) if len(new2[i][1]) > 0]
print(len(mask),len(mask2))

forsurvey=[]
for i in range(len(mask2)-1):
    if len(mask2[i][1]) == 1:
        j = i+1
        if [mask2[i][0],mask2[i][1][0]] != [mask2[j][1][0],mask2[j][0]]:
            forsurvey.append(mask2[i])
    else:
        forsurvey.append(mask2[i])
forsurvey


367 84


[['!1T Family Dental', ['1st Family Dental']],
 ['!2 Technologies', ['12 Technologies']],
 ['# 1 BAR', ['19 Bar']],
 ['#1 CLEANERS', ['$1.25 CLEANERS']],
 ['#1 Cochran', ['1 Cochran']],
 ['#1 Mortgage', ['1st Mortgage']],
 ['#1 SUN TAN AND TRAVEL, INC.', ['#1 SUN TAN & TRAVEL INC.']],
 ['#2 Downing Place', ['2 Downing Place']],
 ['#2 LA NAILS', ['#2 LA NAIL']],
 ['#7 STEAK-N-TAKE RESTAURANT', ['#7 STEAK-N-TAKE RESTARUANT']],
 ['#9 AREA VOLUNTEER FIRE DEPARTMENT', ['#9 AREA VOL. FIRE DEPARTMENT']],
 ['$ 1.49 DRY CLEAN CITY # 9',
  ['$1 69 DRY CLEAN CITY', '$1.69 DRY CLEAN CITY', '$1.50 DRY CLEAN CITY']],
 ['$1 69 DRY CLEAN CITY', ['$1.69 DRY CLEAN CITY', '$1.79 DRY CLEAN CITY']],
 ['$1.00 DOLLAR STORE', ['$1 DOLLAR STORE']],
 ['$1.25 CLEANERS', ['#1 CLEANERS', '$1.25 DRY-CLEANERS']],
 ['$1.25 DRY CLEAN CENTER, INC.',
  ['$1.25 DRY CLEAN SUPER CENTER', '1.25 DRY CLEAN SUPER CENTER']],
 ['$1.25 DRY CLEAN SUPER CENTER',
  ['1.25 DRY CLEAN SUPER CENTER',
   '1.25 DRY CLEAN SUPER CENTER OF',

# Write a .txt file in the format for importing to Qualtrics

The code below can be converted to illustrate the process by which I create the .txt files that are formatted to be imported into Qualtrics. I don't run the code here though because it would take a long time.

In [198]:
maxs = int(math.ceil(len(forsurvey) / 100))
print(maxs)
length = len(forsurvey)

start = 0
counter = 0

for z in range(1,maxs+1):
    end = min(z*100,length)
    submask = forsurvey[start:end]
    start = end+1
    survey_file = open("survey-test-%s" %z +".txt","w")
    survey_file.write("[[AdvancedFormat]] \n\n")
    for j in range(0,len(submask),10):
        survey_file.write("[[Block: Firm Names - %s" % counter + "]]\n")
        for k in range(j,min(j+10,len(submask))):
            survey_file.write(" [[Question:MC:MultipleAnswer:Vertical]] \n \
            %s" % submask[k][0].upper() + "\n\n")
            survey_file.write("  [[AdvancedChoices]] \n")
            for i in range(0,len(submask[k][1])):
                survey_file.write("    [[Choice: %s" % i +"]] \n \
                %s" % submask[k][1][i].upper() +'\n')
            survey_file.write("\n")
        progbar(counter,maxs*10)
        counter += 1
    survey_file.close()

1
60%

In [219]:
crosswalk1 = []
bisids=[]
for i in range(len(forsurvey)):
    crosswalk1.append(forsurvey[i][0])
    bisids.append("bid"+str(i))
    for j in range(len(forsurvey[i][1])):
        crosswalk1.append(forsurvey[i][1][j])
        bisids.append("bid"+str(i))

crosswalk = pd.DataFrame({'cw1' : crosswalk1[:20],
 'bid':bisids[:20]
  })
print(forsurvey[0])
print(forsurvey[0][1])
#print(mask[23][1][0][0])

#inds = []
#for x in range(len(forsurvey)):
#    for i in range(len(forsurvey[x][1])):
#         if mask[23][1][0] in forsurvey[x][1][i]:
#                inds.append(x)
#mask[inds[1]]
#inds

crosswalk

['!1T Family Dental', ['1st Family Dental']]
['1st Family Dental']


In [ ]:
args = 1
name = str('test%s' % args + '.csv')
header = ['bid','cw1']
crosswalk.to_csv(name,sep=',',columns=header,index=False)

In [220]:
%%bash
git add fec-string-prep.ipynb
git status
git commit -m "Updated the cluster process. Created crosswalk .csv file in addition to dramatically reducing the number of surveys created."

On branch master
Your branch is ahead of 'origin/master' by 2 commits.
  (use "git push" to publish your local commits)
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   fec-string-prep.ipynb
	deleted:    survey-test.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/
	survey-test-1.txt
	test.csv
	test1.csv

[master 4b42c6f] Updated the cluster process. Created crosswalk .csv file in addition to dramatically reducing the number of surveys created.
 2 files changed, 291 insertions(+), 4705 deletions(-)
 delete mode 100644 survey-test.txt


The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.


In [ ]:
%%bash
git status